In [1]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from datetime import datetime
import time
import MetaTrader5 as mt5
# Initialize the bound
mt5.initialize()

# Set the symbol
symbol = "XAUUSD"
import pygame

pygame.init()

sound_file = "message-ringtone-magic.mp3"  # Replace with the path to your sound file
pygame.mixer.music.load(sound_file)

# Set filling mode
filling_type = mt5.symbol_info(symbol).filling_mode

# Set the point
point = mt5.symbol_info(symbol).point

username = 'amiramirmashgh'
password = '1234Asdf'

tv = TvDatafeed(username, password)

pygame 2.5.1 (SDL 2.28.2, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


error while signin
you are using nologin method, data you access may be limited


In [2]:

tv = TvDatafeed(username, password)

error while signin
you are using nologin method, data you access may be limited


In [3]:
def get_dat_5min_gold():
    # get data
    df = tv.get_hist(symbol='XAUUSD' ,exchange='OANDA', interval=Interval.in_5_minute,n_bars=400)
    #correct timing
    offset = pd.Timedelta(hours=1)
    df.index = df.index - offset
    return df

def get_dat_5min_eur():
    # get data
    df = tv.get_hist(symbol='EURUSD',exchange='OANDA',interval=Interval.in_5_minute,n_bars=400)
    #correct timing
    offset = pd.Timedelta(hours=1)
    df.index = df.index - offset
    return df


def get_dat_1_hour_gold():
    # get data
    df = tv.get_hist(symbol='XAUUSD',exchange='OANDA',interval=Interval.in_1_hour,n_bars=400)
    #correct timing
    offset = pd.Timedelta(hours=1)
    df.index = df.index - offset
    return df

def get_dat_1_hour_eur():
    # get data
    df = tv.get_hist(symbol='EURUSD',exchange='OANDA',interval=Interval.in_1_hour,n_bars=400)
    #correct timing
    offset = pd.Timedelta(hours=1)
    df.index = df.index - offset
    return df

In [4]:
def analize_data_gold(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')



    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy' 
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'  
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] ) 
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles 
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"  
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)  

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"  
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"  
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1) 

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green') :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red') :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1) 



    df_XUD['SL_sell_shifted1'] = df_XUD['high'].shift(1)
    df_XUD['SL_sell_shifted2'] = df_XUD['high'].shift(2)
    df_XUD['SL_sell_shifted3'] = df_XUD['high'].shift(3)
    df_XUD['SL_sell_shifted4'] = df_XUD['high'].shift(4)

    df_XUD['SL_buy_shifted1'] = df_XUD['low'].shift(1)
    df_XUD['SL_buy_shifted2'] = df_XUD['low'].shift(2)
    df_XUD['SL_buy_shifted3'] = df_XUD['low'].shift(3)
    df_XUD['SL_buy_shifted4'] = df_XUD['low'].shift(4)

    def determine_SL_buy(row):
       return min(row['SL_buy_shifted1'],row['SL_buy_shifted2'],row['SL_buy_shifted3'],row['SL_buy_shifted4']) - 0.1

    def determine_SL_sell(row):
       return max(row['SL_sell_shifted1'],row['SL_sell_shifted2'],row['SL_sell_shifted3'],row['SL_sell_shifted4']) + 0.1

    df_XUD['SL_sell'] = df_XUD.apply(determine_SL_sell, axis=1) 
    df_XUD['SL_buy'] = df_XUD.apply(determine_SL_buy, axis=1) 


    def determine_entry_point_buy(row):
       return row["high"] + 0.1

    def determine_entry_point_sell(row):
       return row["low"] - 0.1



    df_XUD['entry_sell'] =  df_XUD.apply(determine_entry_point_sell, axis=1) 
    df_XUD['entry_buy'] =  df_XUD.apply(determine_entry_point_buy, axis=1) 


    def determine_TP_buy(row):
       return 1.5 * (row["entry_buy"] - row["SL_buy"]) + row["entry_buy"]

    def determine_TP_sell(row):
        return  abs(1.5 * abs(row["SL_sell"] -   row["entry_sell"])    - row["entry_sell"] )


    df_XUD['TP_sell'] = df_XUD.apply(determine_TP_sell, axis=1) 
    df_XUD['TP_buy'] = df_XUD.apply(determine_TP_buy, axis=1) 

    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy" and abs(row["entry_buy"] - row["SL_buy"]) >= 0.5 ):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"  and abs(row["SL_sell"] -   row["entry_sell"]) >= 0.5):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell" and abs(row["SL_sell"] -   row["entry_sell"]) >= 0.5):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy" and abs(row["entry_buy"] - row["SL_buy"]) >= 0.5 ):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1) 

    # Deleting multiple columns
    columns_to_delete = ["SL_buy_shifted4","SL_buy_shifted3","SL_buy_shifted2","SL_buy_shifted1","SL_sell_shifted4","SL_sell_shifted3","SL_sell_shifted2","SL_sell_shifted1","Hist_color_V1", "MACD_Histogram_shifted", "Hist_color_V2" , "Hist_color_V2_shifted","candle_size_shifted4","candle_size_shifted3","candle_size_shifted2","candle_size_shifted1"]
    df_XUD = df_XUD.drop(columns_to_delete, axis=1)

    df_XUD.drop(df_XUD.index[-1], inplace=False)

    # Save the DataFrame to a CSV file
    df_XUD.to_csv(f'gold.csv', index=True)
    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [5]:
def analize_data_eur(df_XUD):
    # Add SMA 20
    window_size = 20
    df_XUD['SMA 20'] = df_XUD['close'].rolling(window=window_size).mean()

    # ADD diff and color off the candle

    # Calculate the difference between current and previous day's closing price
    df_XUD['price_diff_CO'] = df_XUD['close'] -  df_XUD['open']

    # Add a new column to indicate green (1) or red (0)
    df_XUD['candle color'] = df_XUD['price_diff_CO'].apply(lambda x: 'green' if x > 0 else 'red')



    # Assuming your DataFrame is named df
    short_window = 12  # Short-term EMA
    long_window = 26   # Long-term EMA
    signal_window = 9  # Signal line EMA

    # Calculate Short-term EMA
    df_XUD['EMA_short'] = df_XUD['close'].ewm(span=short_window, adjust=False).mean()

    # Calculate Long-term EMA
    df_XUD['EMA_long'] = df_XUD['close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD line
    df_XUD['MACD'] = df_XUD['EMA_short'] - df_XUD['EMA_long']

    # Calculate Signal line (Signal line is typically a 9-period EMA of MACD values)
    df_XUD['Signal_Line'] = df_XUD['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate MACD Histogram
    df_XUD['MACD_Histogram'] = df_XUD['MACD'] - df_XUD['Signal_Line']

    # Determine MACD trend direction (Above/Below Zero Line)
    df_XUD['MACD_Trend'] = df_XUD['MACD'].apply(lambda x: 'Above' if x > 0 else 'Below')

    # Determine Signal_Line trend direction (Above/Below Zero Line)
    df_XUD['Signal_Line_Trend'] = df_XUD['Signal_Line'].apply(lambda x: 'Above' if x > 0 else 'Below')


    # Determine Histogram color (Green if MACD > Signal Line, Red otherwise)
    df_XUD['Hist_color_V1'] = df_XUD.apply(lambda row: 'green' if row['MACD'] > row['Signal_Line'] else 'red', axis=1)


    # Function to determine color based on conditions
    def determine_color(row):
        if row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'green'  # Signal Line is below MACD and Histogram is increasing
        elif row['MACD_Histogram'] > 0 and abs(row['MACD_Histogram']) < abs(row['MACD_Histogram_shifted']):
            return 'light green'  # Signal Line is above MACD and Histogram is decreasing
        elif row['MACD_Histogram'] < 0 and abs(row['MACD_Histogram']) > abs(row['MACD_Histogram_shifted']):
            return 'red'  # Signal Line is above MACD and Histogram is decreasing
        else:
            return 'light red'  # Signal Line is above MACD and Histogram is not decreasing

    # Calculate the shift of the MACD_Histogram column
    df_XUD['MACD_Histogram_shifted'] = df_XUD['MACD_Histogram'].shift(1)

    # Apply the determine_color function to create the color column
    df_XUD['Hist_color_V2'] = df_XUD.apply(determine_color, axis=1)

    # Function to determine color based on conditions
    def determine_color_finisher(row):
        if (row['Hist_color_V2_shifted'] ==  'light green' or row['Hist_color_V2_shifted'] ==  'green') and row['Hist_color_V2'] == "light red":
            return 'red'  # fix first hist candle
        elif (row['Hist_color_V2_shifted'] ==  'light red' or row['Hist_color_V2_shifted'] ==  'red') and row['Hist_color_V2'] == "light green":
            return 'green'  # fix first hist candle
        else:
            return  row['Hist_color_V2'] # return defulat

    # Calculate the shift of the MACD_Histogram column
    df_XUD['Hist_color_V2_shifted'] = df_XUD['Hist_color_V2'].shift(1)

    # Advaned level: GOD bless me
    df_XUD['Hist_color_V3'] = df_XUD.apply(determine_color_finisher, axis=1)


    # Create a cross counter column
    counter = 0
    counter_list = []
    prev_color = None

    for color in df_XUD['Hist_color_V1']:
        if prev_color is None or color == prev_color:
            counter += 1
        else:
            counter = 1
        counter_list.append(counter)
        prev_color = color

    df_XUD['cross_counter'] = counter_list

    def determine_candle_from_sma20(row):
        if (row['SMA 20'] > row['low']  and row['SMA 20'] < row['close']  and row['candle color'] == 'green'):
            return 'buy' 
        elif (row['SMA 20'] > row['close']  and row['SMA 20'] < row['high']  and row['candle color'] == 'red') :
            return 'sell'  
        else:
            return  '' # return defulat

    df_XUD['inside candle (rule 1)'] = df_XUD.apply(determine_candle_from_sma20, axis=1)

    def determine_distance_sma20(row):
        if (row['inside candle (rule 1)'] == 'buy'):
            return (row['high'] - row['SMA 20'] ) 
        elif (row['inside candle (rule 1)'] == 'sell') :
            return row['SMA 20'] - row['low']
        else:
            return  'no position' # return defulat

    df_XUD['distance SMA 20 from high or close'] = df_XUD.apply(determine_distance_sma20, axis=1)


    # from distance where sma 20 get broke and the candle closed , it shouldn't bigger than 4 other candles 
    df_XUD['candle_size'] = df_XUD['high'] - df_XUD['low']
    df_XUD['candle_size_shifted1'] = df_XUD['candle_size'].shift(1)
    df_XUD['candle_size_shifted2'] = df_XUD['candle_size'].shift(2)
    df_XUD['candle_size_shifted3'] = df_XUD['candle_size'].shift(3)
    df_XUD['candle_size_shifted4'] = df_XUD['candle_size'].shift(4)

    def determine_rule2(row):
        if row['distance SMA 20 from high or close'] == 'no position':
            return "False"
        elif (row['distance SMA 20 from high or close'] < row['candle_size_shifted4'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted2'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted3'] or row['distance SMA 20 from high or close'] < row['candle_size_shifted1']):
            return "True"  
        else:
            return  'False'

    df_XUD['rule_2'] = df_XUD.apply(determine_rule2, axis=1)  

    def determine_rule3(row):
        if (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Above') :
            return "buy"
        elif (row['MACD_Trend'] == 'Above'  and row['Signal_Line_Trend'] == 'Below'):
            return "Half_buy"  
        elif (row['MACD_Trend'] == 'Below'  and row['Signal_Line_Trend'] == 'Below'):
            return "sell"  
        else:
            return  'Half_sell'

    df_XUD['rule_3'] = df_XUD.apply(determine_rule3, axis=1) 

    #for long sabz kamrang and for short germez kamrang?
    def determine_rule4_5(row):
        if (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light red' and row['cross_counter'] >= 4  ) :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'light green') :
            return "buy"
        elif (row['inside candle (rule 1)'] == 'buy'  and row['Hist_color_V3'] == 'green' and row['cross_counter'] <= 2 ):
            return "buy"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light green' and row['cross_counter'] >= 4 ) :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'light red') :
            return "sell"
        elif (row['inside candle (rule 1)'] == 'sell'  and row['Hist_color_V3'] == 'red' and row['cross_counter'] <= 2 ):
            return "sell"

        else:
            return  'unknown'

    df_XUD['rule_4_5'] = df_XUD.apply(determine_rule4_5, axis=1) 



    df_XUD['SL_sell_shifted1'] = df_XUD['high'].shift(1)
    df_XUD['SL_sell_shifted2'] = df_XUD['high'].shift(2)
    df_XUD['SL_sell_shifted3'] = df_XUD['high'].shift(3)
    df_XUD['SL_sell_shifted4'] = df_XUD['high'].shift(4)

    df_XUD['SL_buy_shifted1'] = df_XUD['low'].shift(1)
    df_XUD['SL_buy_shifted2'] = df_XUD['low'].shift(2)
    df_XUD['SL_buy_shifted3'] = df_XUD['low'].shift(3)
    df_XUD['SL_buy_shifted4'] = df_XUD['low'].shift(4)

    def determine_SL_buy(row):
       return min(row['SL_buy_shifted1'],row['SL_buy_shifted2'],row['SL_buy_shifted3'],row['SL_buy_shifted4']) - 0.0001

    def determine_SL_sell(row):
       return max(row['SL_sell_shifted1'],row['SL_sell_shifted2'],row['SL_sell_shifted3'],row['SL_sell_shifted4']) + 0.0001

    df_XUD['SL_sell'] = df_XUD.apply(determine_SL_sell, axis=1) 
    df_XUD['SL_buy'] = df_XUD.apply(determine_SL_buy, axis=1) 


    def determine_entry_point_buy(row):
       return row["high"] + 0.0001

    def determine_entry_point_sell(row):
       return row["low"] - 0.0001



    df_XUD['entry_sell'] =  df_XUD.apply(determine_entry_point_sell, axis=1) 
    df_XUD['entry_buy'] =  df_XUD.apply(determine_entry_point_buy, axis=1) 


    def determine_TP_buy(row):
       return 1.5 * (row["entry_buy"] - row["SL_buy"]) + row["entry_buy"]

    def determine_TP_sell(row):
        return  abs(1.5 * abs(row["SL_sell"] -   row["entry_sell"])    - row["entry_sell"] )


    df_XUD['TP_sell'] = df_XUD.apply(determine_TP_sell, axis=1) 
    df_XUD['TP_buy'] = df_XUD.apply(determine_TP_buy, axis=1) 

    def determine_finall_desions(row):
        if (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "buy" and row["rule_4_5"] == "buy" and abs(row["entry_buy"] - row["SL_buy"]) >= 0.0005 ):
            return "buy"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "sell" and row["rule_4_5"] == "sell"  and abs(row["SL_sell"] -   row["entry_sell"]) >= 0.0005):
            return "sell"
        elif (row["candle color"] == "red" and row["inside candle (rule 1)"] == "sell" and  row["rule_2"] == "True" and row["rule_3"] == "Half_sell" and row["rule_4_5"] == "sell" and abs(row["SL_sell"] -   row["entry_sell"]) >= 0.0005):
            return "sell"
        elif (row["candle color"] == "green" and row["inside candle (rule 1)"] == "buy" and  row["rule_2"] == "True" and row["rule_3"] == "Half_buy" and row["rule_4_5"] == "buy" and abs(row["entry_buy"] - row["SL_buy"]) >= 0.0005 ):
            return "buy"
        else :
            return ""

    df_XUD['finall_desions'] = df_XUD.apply(determine_finall_desions, axis=1) 

    # Deleting multiple columns
    columns_to_delete = ["SL_buy_shifted4","SL_buy_shifted3","SL_buy_shifted2","SL_buy_shifted1","SL_sell_shifted4","SL_sell_shifted3","SL_sell_shifted2","SL_sell_shifted1","Hist_color_V1", "MACD_Histogram_shifted", "Hist_color_V2" , "Hist_color_V2_shifted","candle_size_shifted4","candle_size_shifted3","candle_size_shifted2","candle_size_shifted1"]
    df_XUD = df_XUD.drop(columns_to_delete, axis=1)

    df_XUD.drop(df_XUD.index[-1], inplace=False)

    # Save the DataFrame to a CSV file
    df_XUD.to_csv(f'gold.csv', index=True)
    # print("DataFrame saved to 'output.csv'")
    return df_XUD

In [6]:
# SELL LIMIT
def sell_signal(price_entry ,tp ,sl, symbol):
    request_sell_limit = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": 1.0,
        "type": mt5.ORDER_TYPE_SELL_STOP,
        "price": price_entry,
        "tp": tp,
        "sl": sl, 
        "type_filling": filling_type,
        "type_time": mt5.ORDER_TIME_GTC,
    }
    info_order = mt5.order_send(request_sell_limit)
    print("Sell" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)
    if info_order.comment != "Request executed":
        print("WARNINGS", info_order.comment)
    return info_order

# BUY LIMIT
def buy_signal(price_entry ,tp ,sl, symbol):
    request_buy_limit = {
        "action": mt5.TRADE_ACTION_PENDING,
        "symbol": symbol,
        "volume": 1.0,
        "tp": tp,
        "sl": sl, 
        "type": mt5.ORDER_TYPE_BUY_STOP,
        "price": price_entry,
        "type_filling": filling_type,
        "type_time": mt5.ORDER_TIME_GTC,
    }
    info_order = mt5.order_send(request_buy_limit)
    print("Buy" , "entry price :" , price_entry,"TP :" ,tp,"SL :", sl)
    if info_order.comment != "Request executed":
        print("WARNINGS", info_order.comment)
    # if (info_order.comment == "WARNINGS Invalid price")
    return info_order


In [7]:
def extract_signal(df_XUD):
    if df_XUD["finall_desions"][-1] == "buy" or df_XUD["finall_desions"][-1] == "buy_half" :
        return 1
    elif df_XUD["finall_desions"][-1] == "sell" or df_XUD["finall_desions"][-1] == "sell_half" :
        return -1
    else :
        return 0

In [8]:
def duplicate_signal(df_XUD):
    if df_XUD["finall_desions"][-1] == df_XUD["finall_desions"][-2] or df_XUD["finall_desions"][-1] == df_XUD["finall_desions"][-3] or df_XUD["finall_desions"][-1] == df_XUD["finall_desions"][-4]:
        print("dup True")
        return True
    else:
        print("dup False")
        return False
    
    print("dup error")
    return False


In [9]:
def change_candle(df_XUD):
    if df_XUD["inside candle (rule 1)"][-1] == "buy":
        print("inside candle (rule 1) , buy" ,df_XUD["inside candle (rule 1)"][-1] == "buy" )
        if df_XUD["finall_desions"][-2] == "sell" or df_XUD["finall_desions"][-3] == "sell" or df_XUD["finall_desions"][-4] == "sell" :
            print("Oh noooooo")
            return True
        else:
            print("mew0 ohhhh")
            return False
    elif df_XUD["inside candle (rule 1)"][-1] == "sell":
        print("inside candle (rule 1) , sell" ,df_XUD["inside candle (rule 1)"][-1] == "sell" )
        if df_XUD["finall_desions"][-2] == "buy" or df_XUD["finall_desions"][-3] == "buy" or df_XUD["finall_desions"][-4] == "buy" :
            print("Oh noooo")
            return True
        else:
            print("mew0 oh")
            return False
        
    print("dup error")
    return False

In [10]:
def Cancel_Order(info_order):
    # REMOVE PENDING ORDER
    print("Cancel order")
    request = {
         "action" : mt5.TRADE_ACTION_REMOVE ,
          "order" : info_order.order ,
    }
    mt5.order_send(request)


In [12]:
def main(counter):
    while(True):
        
      current_time = datetime.now()
      current_minutes =int(str(current_time.minute))%5
      minutes =int(str(current_time.minute))%5
      current_second = current_time.second
      signal_gold = 0
      signal_eur = 0
      if (minutes == 0) and current_second >= 0 and current_second <= 1:

         df_XUDD = analize_data_gold(get_dat_5min_gold())
         # Check don't have duplicate signal
         if df_XUDD["finall_desions"][-1] != "":
            print("meow",current_time.minute  ,  df_XUDD["finall_desions"][-1] )
            if  duplicate_signal(df_XUDD) == False:
               print("meow1",duplicate_signal(df_XUDD))
               # Create the signals
               if change_candle(df_XUDD) == False:
                  signal_gold =  extract_signal(df_XUDD)
                  print("sssss")
               else :
                  print("mewo")
                  Cancel_Order(info_order)
      if (minutes == 0) and current_second >= 0 and current_second <= 1 and current_minutes == 30:
         df_XUDD = analize_data_gold(get_dat_1_hour_gold())
         # Check don't have duplicate signal
         if df_XUDD["finall_desions"][-1] != "":
            print("meow",current_time.minute  ,  df_XUDD["finall_desions"][-1] )
            if  duplicate_signal(df_XUDD) == False:
               print("meow1",duplicate_signal(df_XUDD))
               # Create the signals
               if change_candle(df_XUDD) == False:
                  signal_gold =  extract_signal(df_XUDD)
                  print("sssss")
               else :
                  print("mewo")
                  Cancel_Order(info_order)



      if (minutes == 0) and current_second >= 0 and current_second <= 1:

         df_EUR = analize_data_eur(get_dat_5min_eur())
         # Check don't have duplicate signal
         if df_EUR["finall_desions"][-1] != "":
            print("meow",current_time.minute  ,  df_EUR["finall_desions"][-1] )
            if  duplicate_signal(df_EUR) == False:
               print("meow1",duplicate_signal(df_EUR))
               # Create the signals
               if change_candle(df_EUR) == False:
                  signal_eur =  extract_signal(df_EUR)
                  print("sssss")
               else :
                  print("mewo")
                  Cancel_Order(info_order)
      if (minutes == 0) and current_second >= 0 and current_second <= 1 and current_minutes == 30:
         df_EUR = analize_data_eur(get_dat_1_hour_eur())
         # Check don't have duplicate signal
         if df_EUR["finall_desions"][-1] != "":
            print("meow",current_time.minute  ,  df_EUR["finall_desions"][-1] )
            if  duplicate_signal(df_EUR) == False:
               print("meow1",duplicate_signal(df_EUR))
               # Create the signals
               if change_candle(df_EUR) == False:
                  signal_eur =  extract_signal(df_EUR)
                  print("sssss")
               else :
                  print("mewo")
                  Cancel_Order(info_order)
      # check open positions
    #is_time=True #Only to run the trading NOW
      if signal_gold != 0:
        symbol = "XAUUSD"
        lot = 1  
        # Create the signals
        pygame.mixer.music.play()
        if signal_gold == 1:
           info_order = buy_signal( df_XUDD["entry_buy"][-1],df_XUDD["TP_buy"][-1],df_XUDD["SL_buy"][-1] , symbol)
           open_order_time = datetime.now()
           print("open sell position" , open_order_time.hour , open_order_time.minute)
        elif signal_gold == -1:
            info_order = sell_signal( df_XUDD["entry_sell"][-1],df_XUDD["TP_sell"][-1],df_XUDD["SL_sell"][-1] , symbol)
            open_order_time = datetime.now()
            print("open sell position" , open_order_time.hour , open_order_time.minute)



      if signal_eur != 0:
        symbol = "EURUSD"
        lot = 1  
        # Create the signals
        pygame.mixer.music.play()
        if signal_eur == 1:
           info_order = buy_signal( df_EUR["entry_buy"][-1],df_EUR["TP_buy"][-1],df_EUR["SL_buy"][-1] , symbol)
           open_order_time = datetime.now()
           print("open sell position" , open_order_time.hour , open_order_time.minute)
        elif signal_eur == -1:
            info_order = sell_signal( df_EUR["entry_sell"][-1],df_EUR["TP_sell"][-1],df_EUR["SL_sell"][-1] , symbol)
            open_order_time = datetime.now()
            print("open sell position" , open_order_time.hour , open_order_time.minute)
        # Run the algorithm
counter = 0
if __name__ == "__main__" :
    main(counter)

meow 35 sell
dup False
dup False
meow1 False
inside candle (rule 1) , sell True
mew0 oh
sssss
Sell entry price : 1.08434 TP : 1.08257 SL : 1.08552
open sell position 19 35
meow 25 sell
dup False
dup False
meow1 False
inside candle (rule 1) , sell True
mew0 oh
sssss
Sell entry price : 1.084535 TP : 1.0829075000000001 SL : 1.08562
open sell position 20 25
meow 45 sell
dup False
dup False
meow1 False
inside candle (rule 1) , sell True
mew0 oh
sssss
Sell entry price : 1.084535 TP : 1.08281 SL : 1.085685
open sell position 20 45
meow 40 buy
dup False
dup False
meow1 False
inside candle (rule 1) , buy True
mew0 ohhhh
sssss
Buy entry price : 1898.145 TP : 1900.2599999999998 SL : 1896.7350000000001
open sell position 21 40
meow 5 buy
dup False
dup False
meow1 False
inside candle (rule 1) , buy True
mew0 ohhhh
sssss
Buy entry price : 1898.3049999999998 TP : 1900.3899999999994 SL : 1896.9150000000002
open sell position 22 5
meow 20 buy
dup True


SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'data.tradingview.com'. (_ssl.c:1002)